In [2]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core import StorageContext

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

## Load models

In [22]:
embed_model = HuggingFaceEmbedding(
    model_name="nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True
)
llm = Ollama(model="qwen2:0.5b", request_timeout=360.0)

rerank = SentenceTransformerRerank(model="colbert-ir/colbertv2.0", top_n=3)

<All keys matched successfully>


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at colbert-ir/colbertv2.0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [30]:
llm.chat(messages="helo")

ValidationError: 1 validation error for LLMChatStartEvent
messages
  value is not a valid list (type=type_error.list)

## inputs

In [19]:
documents = SimpleDirectoryReader("./data/").load_data()
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=40)

## Encode new vector database

In [21]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model,transformations=[text_splitter]
)

## Load existing vector DB

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

## Reranking

In [23]:
query_engine = index.as_query_engine(
    llm=llm, streaming=True, similarity_top_k=2, node_postprocessors=[rerank]
)

## Retrival 

In [26]:
while True:
  query=input("ask: ")
  if query.lower()=="exit":
    break
  streaming_response = query_engine.query(query)
  streaming_response.print_response_stream()


HTTPStatusError: Client error '404 Not Found' for url 'http://localhost:11434/api/chat'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404